<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(fgsm_torchattack.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### FGSM Torchattack

### Lab Objectives:  
- Use a framework to simplify the problem of performing FGSM attack  
- Learn how to use torchattack   
- Build on what we learnt earlier   
- Introduce torchattacks framework   


### Step 1:   
Get the data

In [12]:
# https://arxiv.org/pdf/2010.01950
# https://ar5iv.labs.arxiv.org/html/2010.01950
import torchattacks
import torch
import numpy as np

In [13]:
print(f'Torch version used:  {torch.__version__}')
print(f'Numpy version used:  {np.__version__}')
print(f'Torchattacks version used:  {torchattacks.__version__}')


Torch version used:  2.7.1+cu128
Numpy version used:  2.1.3
Torchattacks version used:  3.3.0


In [14]:
# Setup the device to work with
# This should ensure if there are accelerators in place, such as Apple backend or CUDA, 
# we should be able to take advantage of it.

if torch.cuda.is_available():
    print('Setting the device to cuda')
    device = 'cuda'
elif torch.backends.mps.is_available():
    print('Setting the device to Apple mps')
    device = 'mps'
else:
    print('Setting the device to CPU')
    device = torch.device('cpu')

Setting the device to cuda


In [15]:
# Load the test set
# This is from the Tiny Malware
X_y = np.load(r'../data/tiny_mal_net_X_y_test.npz', allow_pickle=False)
X = X_y['X']
y = X_y['y']

# Get the shape of the dataset
X.shape, y.shape

((1000, 256), (1000,))

In [16]:
# Convert both the feature and targets to torch tensors
# These currently have dimensions (1, 1, 256, 256) -> (batch_size, channels, height, width)
X = torch.tensor(X.reshape(-1, 1, 16, 16), dtype=torch.float32)

# Convert the labels also to torch tensors
y = torch.tensor(y, dtype=torch.long)

### Step 2:  
Load the model  

In [17]:
# Let's load the model we will target
loaded_malware_classifier = torch.jit.load(r'../data/mal_net_tiny_malware_clf.jit', map_location=device)
loaded_malware_classifier.eval()

RecursiveScriptModule(
  original_name=MalClassifier
  (conv_layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=BatchNorm2d)
    (2): RecursiveScriptModule(original_name=ReLU)
    (3): RecursiveScriptModule(original_name=Conv2d)
    (4): RecursiveScriptModule(original_name=BatchNorm2d)
    (5): RecursiveScriptModule(original_name=ReLU)
    (6): RecursiveScriptModule(original_name=Conv2d)
    (7): RecursiveScriptModule(original_name=BatchNorm2d)
    (8): RecursiveScriptModule(original_name=ReLU)
  )
  (global_avg_pool): RecursiveScriptModule(original_name=AdaptiveAvgPool2d)
  (classifier): RecursiveScriptModule(original_name=Conv2d)
)

### Step 3:   
Create the attack via *torchattacks* 

In [18]:
# Setup the torchattack FGSM attack
torchattacks_fgsm_attack = torchattacks.FGSM(model=loaded_malware_classifier, eps=8/255)
torchattacks_fgsm_attack

FGSM(model_name=RecursiveScriptModule, device=cuda:0, return_type=float, attack_mode=default, targeted=False, normalization_used=False, eps=0.03137254901960784)

In [23]:
# Generate the adversarial example
adversarial_examples = torchattacks_fgsm_attack(inputs=X, labels=y)

# Peak into the the first example
adversarial_examples.shape

torch.Size([1000, 1, 16, 16])

### Step 4:   
make predictions on the adversarial examples

In [20]:
# Make the prediction
torchattacks_fgsm_logits = loaded_malware_classifier(adversarial_examples)

# Get the first logits
torchattacks_fgsm_logits[0]

tensor([-0.2855,  0.8150,  2.6080, -7.2436, -1.4449], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [21]:
# Get the predictions from the logits
torchattacks_fgsm_preds = torchattacks_fgsm_logits.argmax(dim=-1)

# Peak into the first 10
torchattacks_fgsm_preds[:10]

tensor([2, 2, 2, 1, 2, 1, 0, 2, 2, 2], device='cuda:0')

### Step 5:  
Calculate the adversarial accuracy  

In [22]:
# Get the adversarial accuracy
(y != torchattacks_fgsm_preds.cpu()).sum()/len(torchattacks_fgsm_preds.cpu())

tensor(0.7280)

### Lab Takeaways:  
- We introduced another framework in *torchattacks*  
- We simplified the issue of the FGSM attack  